In [125]:
import pandas as pd
import os
import re
import numpy as np

In [126]:
doctors1 = pd.read_excel('описание врачей с сайта.xlsx')
doctors2 = pd.read_excel('описание врачей с сайта 2.xlsx')
specializations = pd.read_excel('Специализации 13 нояб П1 и П2  (1).xlsx', sheet_name='Sheet1')

In [127]:
doctors1 = doctors1.append(doctors2, ignore_index=True)

In [128]:
doctors1.head()

,UF_ZAV,UF_WEIGHT,Сайт компании,Телефон компании,Город работы,Внешний код,Источник внешней авторизации,UF_BIO,UF_EXP,UF_MANAGER,...,Имя,Фамилия,E-Mail,Последняя авторизация,ID,UF_SUBGROUP,UF_KIDS,UF_ADULT,UF_YEAR,Дата регистрации
0,нет,NaN,NaN,NaN,NaN,NaN,NaN,<p>Врач первой квалификационной категории.</p>...,NaN,NaN,...,Ольга,Алферова,no@mc21.ru,NaN,73763,общая,Кардиолог-ревматолог / Врач функциональной диа...,NaN,2002.0,13.05.2016 13:12:18
1,нет,NaN,NaN,NaN,NaN,NaN,NaN,"<a name=""savorovskayaes""></a> <p><strong>Дерма...",NaN,NaN,...,Евгения,Саворовская,no@mc21.ru,NaN,82111,Валдай,Дерматолог,Дерматолог,2008.0,04.06.2019 20:20:29
2,нет,NaN,NaN,NaN,NaN,NaN,NaN,"<a name=""savorovskaya""></a> <p>Окончила Санкт-...",NaN,NaN,...,Евгения,Саворовская,no@mc21.ru,NaN,57617,общая,Дерматолог,Дерматолог,2008.0,04.03.2015 14:30:49
3,нет,NaN,NaN,NaN,NaN,NaN,NaN,<p>Денис Александрович закончил Новосибирский ...,NaN,NaN,...,Денис,Томилов,no@mc21.ru,NaN,24824,общая,Отоларинголог,Отоларинголог,2008.0,07.09.2012 14:23:06
4,нет,NaN,NaN,NaN,NaN,NaN,NaN,<p>Высшая квалификационная категория по травма...,NaN,NaN,...,Роман,Гришаев,no@mc21.ru,NaN,87365,общая,Травматолог,Травматолог,2003.0,11.11.2020 16:45:38


In [129]:
def cleanhtml(raw_html):
    cleanr = re.compile(r'<.*?>')
    if raw_html is not np.nan:
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext
    return np.nan

def cleantags(raw_html):
    cleanr = re.compile(r'[a-zA-Z|_]*?#')
    if raw_html is not np.nan:
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext
    return np.nan  
    

In [130]:
doctors1['UF_BIO'] = doctors1['UF_BIO'].apply(cleanhtml).apply(cleantags)
doctors1['UF_BIO2'] = doctors1['UF_BIO2'].apply(cleanhtml).apply(cleantags)

In [131]:
def clean_brackets(string):
    cleanr = re.compile(r'\(.+\)')
    if string is not np.nan:
        cleantext = re.sub(cleanr, '', string)
        return cleantext.lower().strip().replace('ё', 'е')
    return ''

def fio(row):
    return ' '.join([clean_brackets(row['Фамилия']), clean_brackets(row['Имя']), clean_brackets(row['Отчество'])])

In [132]:
doctors1['ФИО описание врачей для СВЯЗКИ'] = doctors1.apply(fio, axis=1)

In [133]:
specializations['ФИО специализации для СВЯЗКИ'] = specializations['Исполнитель'].str.lower().str.strip().str.replace('ё', 'е')

In [134]:
doctors1['ФИО описание врачей для СВЯЗКИ'] = doctors1['ФИО описание врачей для СВЯЗКИ'].replace(dct)

In [135]:
doctors1 = doctors1.merge(specializations, left_on='ФИО описание врачей для СВЯЗКИ', right_on='ФИО специализации для СВЯЗКИ', how='left')

In [136]:
doctors1['UF_USER'].isna().value_counts()

False    520
True      27
Name: UF_USER, dtype: int64

In [121]:
# from difflib import get_close_matches

# d1 = doctors1[doctors1['UF_USER'].isna()]['ФИО описание врачей для СВЯЗКИ']
# s1 = specializations['ФИО специализации для СВЯЗКИ'].to_list()

In [123]:
# dct = {}
# for i in d1:
#     key, val = (i, get_close_matches(i, s1, n=1, cutoff=0.9))
#     if val:
#         dct.setdefault(key, val[0])

In [124]:
dct

{'канавец наталья сергеевна': 'канавец наталия сергеевна',
 'юсупов расул васирпашаевич': 'юсупов расул висирпашаевич',
 'бахолдина наталья викторовна': 'бахолдина наталия викторовна',
 'очилов ринат турсунович': 'очилов ринат турсунпулатович',
 'смирнова юлия геннадьевна': 'смирнова юлия геннадиевна',
 'бойкова инна сергевна': 'бойкова инна сергеевна'}

In [138]:
doctors1.to_excel('Правка описания.xlsx', index=False)

In [139]:
check = pd.read_clipboard()

In [142]:
check

,ФИО описание врачей для СВЯЗКИ,Исполнитель,UF_KIDS,UF_ADULT,иси,Специализация АУ_ОСН,Специализация АУ_ДОП,Специализация ВУ_ОСН,Специализация ВУ_ДОП
0,алферова ольга павловна,Алферова Ольга Павловна,Кардиолог-ревматолог / Врач функциональной диа...,NaN,NaN,Кардиолог детский,"ЭКГ, ЭХОКГ",нет,нет
1,юсупова светлана римовна,Юсупова Светлана Римовна,Врач функциональной диагностики,NaN,NaN,Кардиолог детский,нет,нет,нет
2,образцова галина игоревна,Образцова Галина Игоревна,Кардиолог-ревматолог / Врач функциональной диа...,NaN,NaN,Кардиолог детский,"ЭКГ, ЭХОКГ",нет,нет
3,чикова ирина александровна,Чикова Ирина Александровна,Кардиолог-ревматолог,NaN,NaN,Кардиолог детский,Ревматолог,нет,нет
4,корепанова светлана леонидовна,Корепанова Светлана Леонидовна,Кардиолог-ревматолог,NaN,NaN,Кардиолог детский,"ЭКГ, ЭХОКГ",нет,нет
...,...,...,...,...,...,...,...,...,...
432,шушакова оксана викторовна,Шушакова Оксана Викторовна,NaN,Эндоскопист,NaN,Эндоскопист,нет,нет,нет
433,корнилова анна борисовна,Корнилова Анна Борисовна,Эндоскопист,Эндоскопист,NaN,Эндоскопист,Эндоскопист,нет,нет
434,познякова елена олеговна,Познякова Елена Олеговна,NaN,Эндоскопист,NaN,Эндоскопист,Эндоскопист,нет,нет
435,хочинская ольга юрьевна,Хочинская Ольга Юрьевна,NaN,Эндоскопист,NaN,Эндоскопист,Эндоскопист,нет,нет


In [146]:
def c1(row):
    return str(row['Специализация АУ_ОСН']) in str(row['UF_KIDS'])

def c2(row):
    return str(row['Специализация АУ_ОСН']) in str(row['UF_ADULT'])

In [147]:
check['пров1'] = check.apply(c1, axis=1)
check['пров2'] = check.apply(c2, axis=1)

In [148]:
check.to_excel('пров_спец.xlsx')